In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import NoSuchElementException

from webdriver_manager.chrome import ChromeDriverManager


import pytesseract as tess
tess.pytesseract.tesseract_cmd = r'C:\Users\Hl10a\AppData\Local\Programs\Tesseract-OCR\tesseract.exe'


from PIL import Image
import pandas as pd
import time
import os 

# PATH_DATA = 'data/tvpl0.csv'
# PATH_DATA = 'data/tvpl1.csv'
# PATH_DATA = 'data/tvpl2.csv'
PATH_DATA = 'data/tvpl.csv'

In [2]:
class tvpl_selenium():
    def __init__(self, driver):
        self.driver = driver
        self.df = self.create_df()
        '''
        Max: 21277 văn bản (luật: 502, NĐ: 5285, TT: 15490)
        Use: 10000 văn bản (luật: 420, NĐ: 4300, TT: 5280)
        '''
        self.dict_vb = {
            'type': ['Luật', 'Nghị định', 'Thông tư'],
            'index_type': [10, 11, 23],
            'limit_number': [420, 4300, 5280],
        }
      
    #Đăng nhập tài khoản
    def login(self):
        element = self.driver.find_element('xpath', '//div[@class="box-middle"]/p[2]/span').get_attribute('textContent')
        if element == 'Đăng nhập':
            self.driver.find_element('xpath', '//*[@id="usernameTextBox"]').send_keys("nguyencaotoan")
            self.driver.find_element('xpath', '//*[@id="passwordTextBox"]').send_keys("012345678")
            self.driver.find_element('xpath', '//*[@id="loginButton"]').click()
            time.sleep(0.2)
            button_check = self.driver.find_element('xpath', '//*[@id="logintfrom_w"]/p[1]/span').get_attribute('innerHTML')
            if button_check:
                self.driver.find_element('xpath', '/html/body/div[21]/div[3]/div/button[1]/span').click()

    # Tìm kiếm văn bản theo loại
    def load_search_page(self, index_type, page = 1):
        self.driver.get(f"https://thuvienphapluat.vn/page/tim-van-ban.aspx?&type={index_type}&page={page}")
        WebDriverWait(self.driver, 20).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
        self.antiCaptcha()

    # Tạo dataframe rỗng
    def create_df(self):
        world_table_titles = ['Loại văn bản', 'Số hiệu', 'Cơ quan ban hành', 'Ngày ban hành', 'Nội dung', 'Mối quan hệ']
        df = pd.DataFrame(columns = world_table_titles).astype(str)
        df['Ngày ban hành'] = pd.to_datetime(df['Ngày ban hành'], format='%dd/%mm/%Y')
        return df

    # Xuất DataFrame ra file CSV
    def export_df(self):
        self.df.to_csv(PATH_DATA, encoding='utf-8-sig', index=False)

    # Lấy các thuộc tính ở lược đồ
    def get_Attribute(self):
        WebDriverWait(self.driver, 20).until(EC.presence_of_element_located((By.XPATH, '//*[@id="viewingDocument"]/div[2]/div[2]')))
        number = self.driver.find_element('xpath', '//*[@id="viewingDocument"]/div[2]/div[2]').get_attribute('textContent').strip(' \n ').lower()
        text_type = self.driver.find_element('xpath', '//*[@id="viewingDocument"]/div[3]/div[2]').get_attribute('textContent').strip(' \n ').lower()
        organizer = self.driver.find_element('xpath', '//*[@id="viewingDocument"]/div[5]/div[2]').get_attribute('textContent').strip(' \n ').lower()
        date = self.driver.find_element('xpath', '//*[@id="viewingDocument"]/div[7]/div[2]').get_attribute('textContent').strip(' \n ')
        relationship = self.get_relation_text()

        return {
            'Loại văn bản':     text_type,
            'Số hiệu':          number,
            'Cơ quan ban hành': organizer,
            'Ngày ban hành':    date, 
            'Mối quan hệ': relationship,
        }

    # Lấy các mối quan hệ của văn bản
    def get_relation_text(self):
        items = {
            1:  ['được hướng dẫn','guidedDocument'],
            4:  ['được hợp nhất','DuocHopNhatDocument'],
            7:  ['bị sửa đổi, bổ sung','amendedDocument'],
            10: ['bị đính chính', 'correctedDocument'],
            13: ['bị thay thế','replacedDocument'],
            16: ['dẫn chiếu','referentialDocument'],
            19: ['căn cứ', 'basisDocument'],
        }
        relation_text = {}
        for i in range(1, 20, 3):
            try:
                index = int(self.driver.find_element('xpath', f'//*[@id="cmDiagram"]/table/tbody/tr/td/div[1]/div[{i}]/b').get_attribute('textContent'))
            except NoSuchElementException:
                index = int(self.driver.find_element('xpath', f'//*[@id="cmDiagram"]/table/tbody/tr/td/div[1]/div[{i}]/font/b').get_attribute('textContent'))

            list_vb = []
            if index == 0:
                continue

            for j in range(1, index+1):
                links = self.driver.find_element('xpath', f'//*[@id="{items[i][1]}"]/div/div[{j}]/div/a').get_attribute('textContent')
                list_vb.append(links)
            relation_text[items[i][0]] = list_vb

        return relation_text

    # Lấy nội dung văn bản
    def get_data(self):
        try:
            contents = []
            element_click = WebDriverWait(self.driver, 120).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ctl00_Content_ctl00_spNoiDung"]')))
            element_click.click()
            self.antiCaptcha()
            WebDriverWait(self.driver, 120).until(EC.presence_of_element_located((By.XPATH, '//*[@id="divContentDoc"]/div[2]/div/div/p | //*[@id="divContentDoc"]/div[2]/div/div/div/p')))
            
            lines = self.driver.find_elements('xpath', '//*[@id="divContentDoc"]/div[2]/div/div/p | //*[@id="divContentDoc"]/div[2]/div/div/div/p')
            for line in lines[3:]:
                text = line.get_attribute('textContent').split('\n')
                text = ' '.join(text)
                contents.append(text)

            return '\n\n'.join(contents)
        except:
            pass

    # pass captcha
    def antiCaptcha(self):
        while True:
            _adress = None
            if 'page/check.aspx' in self.driver.current_url:
                _adress = '//*[@id="block-info-3col"]/table/tbody/tr[2]/td[2]/img'
            elif 'page/captcha.aspx' in self.driver.current_url:
                _adress = '//*[@id="ctl00_Content_pnlLoginTemplate"]/div[1]/table/tbody/tr[1]/td[1]/img'
            else:
                break

            if _adress:
                self.driver.refresh()
                try:
                    WebDriverWait(self.driver, 0.5).until(EC.presence_of_element_located((By.ID, "error-dialog-form")))
                    # Nếu tồn tại, click vào nút
                    button = WebDriverWait(self.driver, 0.2).until(
                        EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div[3]/div/button/span"))
                    )
                    button.click()
                except:
                    pass
                WebDriverWait(self.driver, 20).until(EC.presence_of_element_located((By.XPATH, _adress)))
                captcha_img = self.driver.find_element(By.XPATH, _adress)
                captcha_img.screenshot('captcha.png')
                time.sleep(1.5)
                img = Image.open('captcha.png')
                text = tess.image_to_string(img)
                time.sleep(1.5)
                self.driver.find_element('xpath', '//*[@id="ctl00_Content_txtSecCode"]').clear()
                self.driver.find_element('xpath', '//*[@id="ctl00_Content_txtSecCode"]').send_keys(text)
                os.remove('captcha.png')

    # Đóng tab
    def close_tab(self):
        self.driver.close()

    # xử lý 1 văn bản
    def process_1_page(self,index_type, page):  
        self.load_search_page(index_type, page)                                                                    
        for cell in range(1, 21):
            
            try:
                row = {}

                WebDriverWait(self.driver, 30).until(EC.element_to_be_clickable((By.XPATH, f'//*[@id="block-info-advan"]/div[2]/div[{cell}]/div[1]/div[2]/p[1]/a')))
                # chọn văn bản thứ cell
                title_element = self.driver.find_element("xpath", f'//*[@id="block-info-advan"]/div[2]/div[{cell}]/div[1]/div[2]/p[1]/a')
                if any(keyword in title_element.get_attribute('textContent') for keyword in ['Law', 'Decree', 'Circular']):
                    continue
                ActionChains(self.driver).move_to_element(title_element).perform()
                self.antiCaptcha()
                if title_element.is_displayed():
                    title_element.click() 

                self.antiCaptcha()
                # lấy nội dung
                row['Nội dung'] = self.get_data()
                # # click vào tab 'lược đồ'
                WebDriverWait(self.driver, 120).until(EC.presence_of_element_located((By.XPATH, '//span[@id="ctl00_Content_ctl00_spLuocDo"]'))).click()  
                # Thuộc tính khác
                row.update(self.get_Attribute())

                # Kiêm tra nội dung
                if row['Nội dung']:
                    self.df = pd.concat([self.df, pd.DataFrame([row])], ignore_index=True)
                    
                self.load_search_page(index_type, page)
            except :
                pass
         
    # xử lý nhiều văn bản trong 1 loại
    def process_pages(self, index_type, limit_number):
        # page_limit = round(limit_number / 20) + 1
        page = 1   
        tmp = 0
        # Cào từng trang
        while len(self.df) <= limit_number:
            try:
                self.process_1_page(index_type, page)
                value = len(self.df)
                print(f'Trang: {page}\tSố lượng: {value - tmp}\tTổng: {value}')
                tmp = value
                # if page == page_limit:
                #     break
            except:
                print('ERROR!', 'Loại:', index_type, '-','Trang:', page)
            
            if page == 20000:
                break

            page = page+1

        print('trang cuối:',page)
   


In [3]:
options = Options()

options.add_experimental_option("detach", True)
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--aggressive-cache-discard")
options.add_argument("--disable-cache")
options.add_argument("--disable-application-cache")
options.add_argument("--disable-offline-load-stale-cache")
options.add_argument("--disk-cache-size=0")
# options.add_argument("--headless")
options.add_argument("--disable-gpu") 
options.add_argument("--dns-prefetch-disable")
options.add_argument("--no-proxy-server")
options.add_argument("--log-level=3")
options.add_argument("--silent")
options.add_argument("--disable-browser-side-navigation")
options.page_load_strategy = 'normal'

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

driver.get(f"https://thuvienphapluat.vn/")

vb = tvpl_selenium(driver)
vb.login()
time.sleep(1)


In [4]:
# vb.process_pages(10, 420)   # Luật
# vb.process_pages(11, 4300)  # Nghị định
vb.process_pages(23, 5280)  # Thông tư
vb.export_df()
vb.driver.quit()

Trang: 1	Số lượng: 12	Tổng: 12
Trang: 2	Số lượng: 10	Tổng: 22
Trang: 3	Số lượng: 11	Tổng: 33
Trang: 4	Số lượng: 10	Tổng: 43
Trang: 5	Số lượng: 10	Tổng: 53
Trang: 6	Số lượng: 12	Tổng: 65
Trang: 7	Số lượng: 9	Tổng: 74
Trang: 8	Số lượng: 10	Tổng: 84
Trang: 9	Số lượng: 9	Tổng: 93
Trang: 10	Số lượng: 12	Tổng: 105
Trang: 11	Số lượng: 10	Tổng: 115
Trang: 12	Số lượng: 10	Tổng: 125
Trang: 13	Số lượng: 11	Tổng: 136
Trang: 14	Số lượng: 8	Tổng: 144
Trang: 15	Số lượng: 11	Tổng: 155
Trang: 16	Số lượng: 11	Tổng: 166
Trang: 17	Số lượng: 11	Tổng: 177
Trang: 18	Số lượng: 11	Tổng: 188
Trang: 19	Số lượng: 9	Tổng: 197
Trang: 20	Số lượng: 11	Tổng: 208
Trang: 21	Số lượng: 11	Tổng: 219
Trang: 22	Số lượng: 10	Tổng: 229
Trang: 23	Số lượng: 10	Tổng: 239
Trang: 24	Số lượng: 12	Tổng: 251
Trang: 25	Số lượng: 8	Tổng: 259
Trang: 26	Số lượng: 9	Tổng: 268
Trang: 27	Số lượng: 12	Tổng: 280
Trang: 28	Số lượng: 9	Tổng: 289
Trang: 29	Số lượng: 10	Tổng: 299
Trang: 30	Số lượng: 14	Tổng: 313
Trang: 31	Số lượng: 8	Tổng: 321
Tra

In [5]:
df1 = pd.read_csv('data/source_raw/tvpl0.csv', encoding='utf-8-sig')
df2 = pd.read_csv('data/source_raw/tvpl1.csv', encoding='utf-8-sig')
df3 = pd.read_csv('data/source_raw/tvpl2.csv', encoding='utf-8-sig')

PATH_DATA = 'data/tvpl.csv'
combined_df = pd.concat([df1, df2, df3], ignore_index=True)
combined_df.to_csv(PATH_DATA, encoding='utf-8-sig', index=False)

In [6]:
combined_df.head()

,Loại văn bản,Số hiệu,Cơ quan ban hành,Ngày ban hành,Nội dung,Mối quan hệ
0,luật,31/2024/qh15,quốc hội,18/01/2024,"<i>Căn cứ <a name=""tvpllink_khhhnejlqt"" href=""...","{'bị sửa đổi, bổ sung': ['Luật Đầu tư 2020', '..."
1,luật,32/2024/qh15,quốc hội,18/01/2024,"<i>Căn cứ <a name=""tvpllink_khhhnejlqt"" href=""...",{'bị thay thế': ['Luật các tổ chức tín dụng 20...
2,luật,29/2023/qh15,quốc hội,28/11/2023,"<i>Căn cứ <a name=""tvpllink_khhhnejlqt"" href=""...","{'bị sửa đổi, bổ sung': ['Luật Đầu tư 2020'], ..."
3,luật,30/2023/qh15,quốc hội,28/11/2023,"<i>Căn cứ <a name=""tvpllink_khhhnejlqt"" href=""...","{'bị sửa đổi, bổ sung': ['Luật Công an nhân dâ..."
4,luật,26/2023/qh15,quốc hội,27/11/2023,"<i>Căn cứ <a name=""tvpllink_khhhnejlqt"" href=""...","{'bị sửa đổi, bổ sung': ['Luật cư trú 2020', '..."
